# Setting Everything Up

In [1]:
import pandas as pd
from datetime import datetime

In [2]:
#import the file from the web
import requests
from bs4 import BeautifulSoup
import pandas as pd
from unicodedata import normalize
import re
from datetime import datetime
from pandas import read_csv 
from sklearn.feature_extraction.text import CountVectorizer
import json
from flatten_json import flatten


# I can give a number or use None to remove maximum ceiling & display all columns
pd.options.display.max_columns = None

# # I want to be able to see the entire narrative, so remove the maximum width for each column
# pd.options.display.max_colwidth = None

# pd.options.display.float_format = '{:,.0f}'.format

import string

%matplotlib inline

In [3]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (8, 8)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [4]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [5]:
%%R

# My commonly used R imports

require('tidyverse')

R[write to console]: Loading required package: tidyverse



── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [6]:
#To Download PDFs
from urllib.parse import urljoin

In [7]:
#To import camelot and PDF-related items
import camelot
import ghostscript
import sys

In [8]:
%%R

library(jpeg)
library(wordcloud)
library(RColorBrewer)
library(wordcloud2)
library(tm)


R[write to console]: Loading required package: RColorBrewer

R[write to console]: Loading required package: NLP

R[write to console]: 
Attaching package: ‘NLP’


R[write to console]: The following object is masked from ‘package:ggplot2’:

    annotate




In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
%%R
library(vistime)
library(shiny)
library(plotly)
#for timeline plotting

R[write to console]: 
Attaching package: ‘plotly’


R[write to console]: The following object is masked from ‘package:ggplot2’:

    last_plot


R[write to console]: The following object is masked from ‘package:stats’:

    filter


R[write to console]: The following object is masked from ‘package:graphics’:

    layout




In [11]:
from collections import Counter


In [62]:
# for playwright stuff
import time
from playwright.async_api import async_playwright
import asyncio
import nest_asyncio


 # Getting 10K

# Putting together code for finding the 10k link

#### Reading the data

In [14]:
# read the dataframe prepared

df_stock_top_us_plus_sec_titles=pd.read_csv("df_stock_top_us_plus_sec_titles.csv")

In [15]:
# this is final


for one_title_searchable in df_stock_top_us_plus_sec_titles['title_searchable']:
    #### ON THE SEC HOME PAGE
    # going to sec page
    sec_home_playwright = await async_playwright().start()
    sec_home_browser = await sec_home_playwright.chromium.launch(headless = False)
    sec_home_page = await sec_home_browser.new_page()

    await sec_home_page.goto("https://www.sec.gov/edgar/searchedgar/companysearch")
    time.sleep(6)
    
    try:

        # input company name (here is where we can enter title variable from df from csv)

        await sec_home_page.fill("xpath=/html/body/div[1]/div/div/div/div[3]/div/div[2]/div[2]/div/div/article/div/div/div[1]/div[2]/div/div/div/div[1]/form/div[2]/input", one_title_searchable)
        time.sleep(6)

        # click search

        await sec_home_page.click("xpath=/html/body/div[1]/div/div/div/div[3]/div/div[2]/div[2]/div/div/article/div/div/div[1]/div[2]/div/div/div/div[1]/form/div[2]/button")
        time.sleep(6)

        #============
        # ON THE NEW PAGE RESULTING WITH ALL COMPANY DOCS
        #input 10K for search
        await sec_home_page.fill("xpath=/html/body/div[4]/div[2]/form/table/tbody/tr[3]/td[1]/input", "10-K")
        time.sleep(6)

        # click search

        await sec_home_page.click("xpath=/html/body/div[4]/div[2]/form/table/tbody/tr[3]/td[5]/input[1]")
        time.sleep(6)


        #get all 10K entries
        list_of_all_ten_k_entries_response= await sec_home_page.content()

        list_of_all_ten_k_entries_doc = BeautifulSoup(list_of_all_ten_k_entries_response, 'html.parser')

        #get the first 10K
        list_of_all_ten_k_entries_table = list_of_all_ten_k_entries_doc.find('table', class_='tableFile2')

        ten_k_entries_rows = list_of_all_ten_k_entries_table.find_all('tr')  # get the first row because it has the most recent 10K

        most_recent_ten_k_link = None

        for one_ten_k_entries_row in ten_k_entries_rows:
            one_ten_k_entries_row_cells = one_ten_k_entries_row.find_all('td')  # Assuming each cell is represented by a <td> tag
        #     print(one_ten_k_entries_row_cells)

            for one_ten_k_entries_row_cell in one_ten_k_entries_row_cells:
        #         print(one_ten_k_entries_row_cell)
                if one_ten_k_entries_row_cell.text.strip() == '10-K':
                    ten_k_document_link_href = one_ten_k_entries_row_cell.find_next('td').find('a')['href']

                    ten_k_document_link_base = "https://www.sec.gov/"
                    ten_k_document_link_tail = ten_k_document_link_href
                    ten_k_document_link = ten_k_document_link_base + ten_k_document_link_tail

        #             print(document_link)

                    # Store the first 10-K link and break out of the loop
                    most_recent_ten_k_link = ten_k_document_link
                    break

            if most_recent_ten_k_link is not None:
                break


        # ONCE I HAVE THE TEN K DOC LINK 
        # 3. Now on the page with the document and graphics table

        ten_k_doc_link_playwright = await async_playwright().start()
        ten_k_doc_link_browser = await ten_k_doc_link_playwright.chromium.launch(headless = False)
        ten_k_doc_link_page = await ten_k_doc_link_browser.new_page()

        time.sleep(6)

        await ten_k_doc_link_page.goto(most_recent_ten_k_link)
        time.sleep(6)

        

        # Initiate the html and beautiful soup content 

        list_of_doc_and_graphics_response= await ten_k_doc_link_page.content()

        list_of_doc_and_graphics_soup_doc = BeautifulSoup(list_of_doc_and_graphics_response, 'html.parser')

        # get actual_full_ten_k_document_link
        list_of_doc_and_graphics_table = list_of_doc_and_graphics_soup_doc.find('table', class_='tableFile')


        list_of_doc_and_graphics_rows = list_of_doc_and_graphics_table.find_all('tr')  # Assuming each row is represented by a <tr> tag

        for list_of_doc_and_graphics_row in list_of_doc_and_graphics_rows:
            list_of_doc_and_graphics_cells = list_of_doc_and_graphics_row.find_all('td')  # Assuming each cell is represented by a <td> tag
        #     print(cells)
            for list_of_doc_and_graphics_cell in list_of_doc_and_graphics_cells:
                if list_of_doc_and_graphics_cell.text.strip() == '10-K':
                    actual_document_link = list_of_doc_and_graphics_cell.find_next('td').find('a')
                    if actual_document_link is not None:
                        actual_ten_k_document_href = actual_document_link['href']
        #                 print("Documents href:", document_href)
                        actual_ten_k_document_link_base="https://www.sec.gov/"
                        actual_ten_k_document_link_tail=actual_ten_k_document_href
                        actual_full_ten_k_document_link=actual_ten_k_document_link_base+actual_ten_k_document_link_tail
                        print(actual_full_ten_k_document_link)
                        print("#########")

                        # Add the values to the DataFrame
                        df_stock_top_us_plus_sec_titles.loc[df_stock_top_us_plus_sec_titles['title_searchable'] == one_title_searchable, 'ten_k_document_link'] = ten_k_document_link
                        df_stock_top_us_plus_sec_titles.loc[df_stock_top_us_plus_sec_titles['title_searchable'] == one_title_searchable, 'actual_full_ten_k_document_link'] = actual_full_ten_k_document_link
    except:
        pass
 
    finally:
        # Close the page and browser
        await ten_k_doc_link_page.close()
        await ten_k_doc_link_browser.close()
        await ten_k_doc_link_playwright.stop()

        # Close the sec_home page and browser
        await sec_home_page.close()
        await sec_home_browser.close()
        await sec_home_playwright.stop()

https://www.sec.gov//ix?doc=/Archives/edgar/data/320193/000032019322000108/aapl-20220924.htm
#########
https://www.sec.gov//ix?doc=/Archives/edgar/data/789019/000156459022026876/msft-10k_20220630.htm
#########
https://www.sec.gov//ix?doc=/Archives/edgar/data/1652044/000165204423000016/goog-20221231.htm
#########
https://www.sec.gov//ix?doc=/Archives/edgar/data/1018724/000101872423000004/amzn-20221231.htm
#########
https://www.sec.gov//ix?doc=/Archives/edgar/data/1045810/000104581023000017/nvda-20230129.htm
#########
https://www.sec.gov//ix?doc=/Archives/edgar/data/1318605/000095017023001409/tsla-20221231.htm
#########
https://www.sec.gov//ix?doc=/Archives/edgar/data/1326801/000132680123000013/meta-20221231.htm
#########
https://www.sec.gov//ix?doc=/Archives/edgar/data/1403161/000140316122000081/v-20220930.htm
#########
https://www.sec.gov//ix?doc=/Archives/edgar/data/59478/000005947823000082/lly-20221231.htm
#########
https://www.sec.gov//ix?doc=/Archives/edgar/data/731766/000073176623

In [17]:
# df_stock_top_us_plus_sec_titles.to_csv("df_stock_top_us_plus_sec_titles_with_links.csv")

In [14]:
# df_stock_top_us_plus_sec_titles=pd.read_csv("df_stock_top_us_plus_sec_titles_with_links.csv")

In [17]:
df_stock_top_us_plus_sec_titles=pd.read_csv("df_stock_top_us_plus_sec_titles_with_links_after_copy_paste_w_python_calc.csv")
df_stock_top_us_plus_sec_titles

,Unnamed: 0.1,to_freeze_name,market_cap_rank,Unnamed: 2,Unnamed: 0,name,american_progress_list,market_cap,price,cik_str,ticker,title_searchable,title_confirmed,tech_or_non_tech,ten_k_document_link,actual_full_ten_k_document_link,most_recent_tax_benefits_from_stock_based_compensation_over_net_income,most_recent_tax_benefits_from_stock_based_compensation,tax_benefits_from_stock_based_compensation_2023,tax_benefits_from_stock_based_compensation_2022,tax_benefits_from_stock_based_compensation_2021,tax_benefits_from_stock_based_compensation_2020,tax_benefits_note,most_recent_stock_or_share_based_compensation,most_recent_stock_or_share_based_compensation_expounded,stock_or_share_based_compensation_end_of_2023,stock_or_share_based_compensation_end_of_2022,stock_or_share_based_compensation_end_of_2021,stock_or_share_based_compensation_end_of_2020,stock_notes,most_recent_tax_rates_low_or_above_ten,most_recent_tax_rates_low_or_above_ten_expounded,most_recent_tax_rates,most_recent_tax_rates_expounded,tax_rates_2023,tax_rates_2022,tax_rates_2021,tax_rates_2020,most_recent_net_income,most_recent_net_income_expounded,net_income_2023,net_income_2022,net_income_2021,net_income_2020,stock_based_compensation_as_a_portion_of_net_income,python_calc_stock_based_compensation_as_a_portion_of_net_income_pct,python_calc_most_recent_tax_benefits_from_stock_based_compensation_over_net_income_pct
0,0,Apple,1,0,0,Apple,absent,$3.030 T,$191.33,320193,AAPL,Apple Inc.,apple inc. (aapl),tech,https://www.sec.gov//Archives/edgar/data/32019...,https://www.sec.gov//ix?doc=/Archives/edgar/da...,-4.0,-4002.0,NaN,"-4,002","-4,056","-2,476",NaN,9280.0,"9,280",NaN,"9,280","8,108","6,975",NaN,above_ten,above_ten,16.2,16.2,NaN,16.2,13.3,14,99803.0,"99,803",NaN,"99,803","94,680","57,411",9,9.298318,-4.009900
1,1,Microsoft,2,1,1,Microsoft,present,$2.533 T,$338.15,789019,MSFT,MICROSOFT CORP,NaN,tech,https://www.sec.gov//Archives/edgar/data/78901...,https://www.sec.gov//ix?doc=/Archives/edgar/da...,2.0,1293.0,NaN,"1,293","1,065",938,NaN,7502.0,"7,502",NaN,"7,502","6,118","5,289",NaN,above_ten,above_ten,13.1,13.1,NaN,13.1,13.8,17,72738.0,"72,738",NaN,"72,738","61,271","44,281",10,10.313729,1.777613
2,2,Alphabet (Google),3,2,2,Alphabet (Google),absent,$1.553 T,$122.63,1652044,GOOG,Alphabet Inc.,NaN,tech,https://www.sec.gov//Archives/edgar/data/16520...,https://www.sec.gov//ix?doc=/Archives/edgar/da...,5.0,2700.0,NaN,"2,700","3,100","3,900",NaN,19362.0,"19,362",NaN,"19,362","15,376","12,991",NaN,above_ten,above_ten,15.9,15.9,NaN,15.9,16.2,NaN,59972.0,"59,972",NaN,"59,972","76,033",NaN,32,32.285066,4.502101
3,3,Amazon,4,3,3,Amazon,present,$1.337 T,$130.38,1018724,AMZN,AMAZON COM INC,NaN,tech,https://www.sec.gov//Archives/edgar/data/10187...,https://www.sec.gov//ix?doc=/Archives/edgar/da...,9.0,1900.0,NaN,"1,900","2,700","4,300",NaN,19621.0,"19,621",NaN,"19,621",12757,9208,NaN,NaN,NaN,NaN,unclear,NaN,unclear,unclear,unclear,21331.0,"21,331",NaN,"21,331","33,364","-2,722",92,91.983498,8.907224
4,4,NVIDIA,5,4,4,NVIDIA,absent,$1.045 T,$423.17,1045810,NVDA,NVIDIA CORP,NaN,tech,https://www.sec.gov//Archives/edgar/data/10458...,https://www.sec.gov//ix?doc=/Archives/edgar/da...,-7.0,-309.0,-309.0,-337,-136,NaN,NaN,2709.0,2709,NaN,"2,709","2,004","1,397",NaN,NaN,NaN,NaN,"Furthermore, the tax effects of accounting for...",NaN,"Furthermore, the tax effects of accounting for...",NaN,NaN,4368.0,"4,368","4,368","9,752","4,332",NaN,62,62.019231,-7.074176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,Regeneron Pharmaceuticals,96,95,95,Regeneron Pharmaceuticals,absent,$78.75 B,$717.83,872589,REGN,"REGENERON PHARMACEUTICALS, INC.",NaN,tech,https://www.sec.gov//Archives/edgar/data/87258...,https://www.sec.gov//ix?doc=/Archives/edgar/da...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,725.0,725,NaN,725,601.7,432,NaN,above_ten,above_ten,10.7,10.7,NaN,10.7,13.4,7.80,4

# [Creating a scraper for cash flows statement for SEC filings.](https://www.sec.gov/)

This is a project in which I automate the extraction of details from the cash flow statements of different companies for SEC filings .


## Getting the Data

My universe of data for this project is the companies with the highest market capitalisation in the United States as of Wednesday, 5th July 2023. 

### Preparing the Data
Here is [a notebook where you can follow along](preparing_top_company_plus_sec_titles_data.ipynb) to see the details for this section.
#### Here is the [source](https://companiesmarketcap.com/) for the 100 companies at the top of this list

* I used Playwright to extract this list since requests produced an error regarding automated scraping. 

### In order to use the data to access SEC filings, it would be easier to have the titles as listed on SEC. Therefore, I found the formal titles on SEC's site

#### Here is the [source](https://www.sec.gov/files/company_tickers.json) for all public companies' details. 
To get the json, I simply copy-pasted the content and assigned the variable since that was faster than getting around site blockers. I converted the json to a dataframe with the following columns:
* Company CIK
* Company ticker
* Company title

### Merging the Data
Since the names in the top 100 list are slightly different from the official SEC titles, I use fuzzymatching to merge the lists on the appropriate columns, resulting in a top 100 list which includes the titles that are more searchable on the SEC website

```
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Define a function to find the best fuzzy match for each title
def find_best_match(title):
    choices = df_company_tickers_transposed['title'].tolist()  # Update column name to 'title'
    best_match, score = process.extractOne(str(title), choices)  # Convert title to string
    if score >= 80:  # Adjust the threshold as needed
        return best_match
    else:
        return None

# Apply the function to find the best matches for each title in df_large_companies_us_soup_doc_table
df_large_companies_us_soup_doc_table['best_match'] = df_large_companies_us_soup_doc_table['Name'].apply(find_best_match)

# Merge the dataframes based on the best matches
merged_df = pd.merge(df_large_companies_us_soup_doc_table, df_company_tickers_transposed, left_on='best_match', right_on='title', how='inner')  # Update column name to 'title'

# Remove the intermediate 'best_match' column
merged_df.drop('best_match', axis=1, inplace=True)

# Print the merged dataframe
print(merged_df)

```

### Now to scrape the data

There are two major processes to figure out for the scraper to work:
1) Get a list of the links of the most recent 10K documents
2) Go into each 10K document link and withdraw the stock-based compensation details

Here is a video showing the goal:

# Get a list of the links of the most recent 10K documents
Here is [a notebook where you can follow along](for_10_K_get_links.ipynb) to see the details for this section.

ON THE SEC PAGE
1) Go to the SEC Home Page
2) Enter the title as retrieved from SEC's website
3) Click search

ON THE PAGE RESULTING FROM SEARCH OF SPECIFIC COMPANY
1) Search for 10-K using playwright
2) Use requests to read resulting page
3) Retrieve the first 10-K Document link (the first is the most recent one)
4) Use playwright to go to the link resulting. It has the most recent 10-K document and graphics

ON THE PAGE RESULTING FROM LINK OF MOST RECENT 10-K DOCUMENT AND GRAPHICS 
1) Retrieve content using requests
2) Extract the link that is specific to the document not the graphics

![guide](guide_for_scraping_links.gif)

# Go into each 10K document link and withdraw the stock-based compensation details
Here is [a notebook where you can follow along](for_10_K_get_links.ipynb) to see the details for this section.

Here, there are two major goals:
1) Navigate to the cash flows statements table
2) Extract the details of the stock-based compensation expense 
* For reference, here is an example of Apple's [most recent 10-K Document](https://www.sec.gov//ix?doc=/Archives/edgar/data/320193/000032019322000108/aapl-20220924.htm) for reference
<insert picture>

ON THE 10K DOCUMENT LINK 

1) Await loading by including a time delay
2) Scroll down the page to ensure that it all loads
3) Only after that is it then possible to use requests to extract the content

## Navigate to the cash flows statements table
1) Usually the cash flows statements have the following titles:
* CONSOLIDATED STATEMENTS OF CASH FLOWS
* CASH FLOWS STATEMENTS
I used this information to trace the table. 
> This extraction did not always work, presumably because sometimes the words were used earlier in the document rather than as a title for the table. One way could be checking if something that often appears reliably is present in the table extracted eg Net income and loss and if not to proceed to the next instance of the word. (Would not use stock-based compensation detail because it does not always appear eg for companies where this is not relevant)
2) In this current iteration, I find the title then find the next table appearing. 

## Extract the details of the stock-based compensation expense 
1) Usually the row with stock-based compensation details will appear with the following title cell:
* Stock-based compensation expense
* Share-based compensation expense
* Stock-based compensation
* Share-based compensation
I used this information to find the table cell which has the stock-related details I want. 
2) I go to the parent of the cell, the table row and extract all table cells in the row
3) I extract all values in the row
4) To get the values, I create a mini-list within all extracted values which includes only the cells with numbers (Some are empty, others are just the % or $. Usually I get three values, usually for the last 3 years)






3) Click search

ON THE PAGE RESULTING FROM SEARCH OF SPECIFIC COMPANY
1) Search for 10-K using playwright
2) Use requests to read resulting page
3) Retrieve the first 10-K Document link (the first is the most recent one)
4) Use playwright to go to the link resulting. It has the most recent 10-K document and graphics

ON THE PAGE RESULTING FROM LINK OF MOST RECENT 10-K DOCUMENT AND GRAPHICS 
1) Retrieve content using requests
2) Extract the link that is specific to the document not the graphics

<insert video>
The 

# Analysis of stock-based compensation expenses and taxes
## How I selected the universe of data
I selected the top 100 companies by market capitalisation to have a defined dataset.
Since I am interested in public companies which offered significant stock-based compensation expenses and whether there was a relationship to the size of their effective tax rate, it would be interesting to analyse all the public companies instead. However, the process required to manage the automatic and manual collection of such data would require more time than the initial scope of this project allowed. 

## Guide for data collected
##### Net income: 
I collected this because I felt that stock-based compensation expenses would make the most sense if they were normalised over net income.
##### Effective tax rate: 
I collected this in order to evaluate if there is any significant relationship between stock-based compensation expenses and tax rates.
##### Income tax benefit related to share-based compensation expense: 
I collected this where it was visible in the 10K because it was the way to see the direct impact of share-based compensation expense.

Here is the [CSV](df_stock_top_us_plus_sec_titles_with_links_after_copy_paste.csv) with all the data that was collected

# Statistical Analysis
# T Test
Here is [a notebook where you can follow along](sec_t_test_analysis.ipynb) to see the details for this section.

Question:
## Are effective tax rates for companies below or above ten depending on the stock compensation expense?
#### Null hypothesis:
There is no difference between the stock-based compensation expenses for companies where the effective tax rates are above or below 10%
#### Plotting
![plotting](plotting_effective_tax_rates_vs_stock.png)
In this t-test, I test whether there is a difference between the effective tax rates of companies are below or above ten depending on the stock-based compensation expense.
* The p-value in this case is large at 0.127, indicating there is no statistically significant difference between the stock-based compensation expense depending on whether the effective tax rates of companies are below or above ten. This is a conclusion that can be supported by the initial plotting of the data which showed the data may be randomly distributed.

Question:
## Normalised over net income, are effective tax rates for companies below or above ten depending on the tax benefits from stock compensation expense?
#### Null hypothesis:
Normalised over net income, there is no difference between the tax benefits from stock compensation expense for companies where the effective tax rates are above or below 10%
* The tax benefits from stock-based compensation expense are not associated with whether or not the effective tax rate is above ten. Effective rates are not likely to be above or below ten depending on tax benefits from stock-based compensation expense.
* The p-value in this case is large at 0.4807, indicating there is no statistically significant difference

# Linear regressions
I ran linear regressions on the data to see if there are any patterns in the continuous data:


#### Investigating impact of net income on stock-based compensation expenses
This result is slightly predictable:
* In summary, this linear regression model suggests that there is a statistically significant relationship between python_calc_stock_based_compensation_as_a_portion_of_net_income_pct and most_recent_tax_rates. The model explains approximately 13% of the variance in most_recent_tax_rates, and the effect of stock_based_compensation_as_a_portion_of_net_income on most_recent_tax_rates is positive and statistically significant.


#### Investigating impact of a company's tech or non-tech status on effective tax rates
*The p-value for this coefficient is 0.0541, which is marginally above the conventional significance level of 0.05. This means that the evidence for a significant difference in tax rates between tech and non-tech companies is not strong enough to be conclusive at a standard significance level. In summary, the linear regression model suggests that tech companies may have lower tax rates on average, but the evidence for this difference is not statistically significant at the conventional significance level. Additional data or further analysis might be needed to draw more definitive conclusions.

#### Investigating impact of a company's tech or non-tech status on effective tax rates
* The coefficients represent the estimated intercept and slope (or effect) of the stock_based_compensation_as_a_portion_of_net_income variable on most_recent_tax_rates: 0.03789
* The multiple R-squared represents the proportion of the variance explained by the model: just 13%. 
* In summary, this linear regression model suggests that there is a statistically significant relationship between python_calc_stock_based_compensation_as_a_portion_of_net_income_pct and most_recent_tax_rates. The model explains approximately 13% of the variance in most_recent_tax_rates, and the effect of stock_based_compensation_as_a_portion_of_net_income on most_recent_tax_rates is positive and statistically significant.